In [153]:
import pandas as pd
import numpy
import os
from torchtext import datasets
from torchtext.legacy import data

- In this competition, you’ll identify specific clinical concepts in patient notes. 
- Specifically, you'll develop an automated method to map clinical concepts from an exam rubric (e.g., “diminished appetite”) to various ways in which these concepts are expressed in clinical patient notes written by medical students (e.g., “eating less,” “clothes fit looser”). Great solutions will be both accurate and reliable.

- If successful, you'll help tackle the biggest practical barriers in patient note scoring, making the approach more transparent, interpretable, and easing the development and administration of such assessments. As a result, medical practitioners will be able to explore the full potential of patient notes to reveal information relevant to clinical skills assessment.

In [157]:
train = pd.read_table('/Users/suhancho/data/kaggle/nbme/train.csv',sep=',')
test = pd.read_table('/Users/suhancho/data/kaggle/nbme/test.csv',sep=',')
features = pd.read_table('/Users/suhancho/data/kaggle/nbme/features.csv',sep=',')
notes = pd.read_table('/Users/suhancho/data/kaggle/nbme/patient_notes.csv',sep=',')
sub = pd.read_table('/Users/suhancho/data/kaggle/nbme/sample_submission.csv',sep=',')

In [158]:
test_merged = pd.merge(test,features,on=['case_num','feature_num'])

In [159]:
merged = pd.merge(train,features,on=['case_num','feature_num'])

In [160]:
# merged['annotation'] = merged['annotation'].str.replace("[]",'1000')
merged['annotation'] = merged['annotation'].str.replace('[','').str.replace(']','')
merged['annotation'] = merged['annotation'].str.replace("'",'')
# merged = merged.replace(r'', 1000)

/var/folders/3m/68621g4n7pd2p55cvf4c9tmh0000gn/T/ipykernel_17476/123955389.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged['annotation'] = merged['annotation'].str.replace('[','').str.replace(']','')


In [239]:
test

,id,case_num,pn_num,feature_num
0,00016_000,0,16,0
1,00016_001,0,16,1
2,00016_002,0,16,2
3,00016_003,0,16,3
4,00016_004,0,16,4


In [238]:
train.head(3)

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']


In [242]:
notes.head(3)

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...


In [241]:
merged.head(30)

,id,case_num,pn_num,feature_num,annotation,location,feature_text
0,00016_000,0,16,0,dad with recent heart attcak,['696 724'],Family-history-of-MI-OR-Family-history-of-myoc...
1,00041_000,0,41,0,,[],Family-history-of-MI-OR-Family-history-of-myoc...
2,00046_000,0,46,0,father: heart attack,['824 844'],Family-history-of-MI-OR-Family-history-of-myoc...
3,00082_000,0,82,0,Father MI,['622 631'],Family-history-of-MI-OR-Family-history-of-myoc...
4,00100_000,0,100,0,Dad-MI,['735 741'],Family-history-of-MI-OR-Family-history-of-myoc...
5,00161_000,0,161,0,father had acute MI,['601 620'],Family-history-of-MI-OR-Family-history-of-myoc...
6,00188_000,0,188,0,father heart attach,['716 735'],Family-history-of-MI-OR-Family-history-of-myoc...
7,00211_000,0,211,0,Father MI,['682 688;695 697'],Family-history-of-MI-OR-Family-history-of-myoc...
8,00224_000,0,224,0,dad had recent MI,['483 500'],Family-history-of-MI-OR-Family-history-of-myoc...
9,00225_000,0,225,0,dad had heart attack,['735 755'],Family-history-of-MI-OR-Family-history-of-myoc...


In [231]:
notes

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...
...,...,...,...
42141,95330,9,Ms. Madden is a 20 yo female presenting w/ the...
42142,95331,9,A 20 YO F CAME COMPLAIN A DULL 8/10 HEADACHE T...
42143,95332,9,Ms. Madden is a 20yo female who presents with ...
42144,95333,9,Stephanie madden is a 20 year old woman compla...


In [205]:
str(notes[(notes.pn_num==16) & (notes.case_num==0)]['pn_history'].tolist()[0])[696:724]

'dad with recent heart attcak'

In [207]:
str(notes[(notes.pn_num==16) & (notes.case_num==0)]['pn_history'].tolist()[0])

'HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. \r\nPMHx: none\r\nRx: uses friends adderrall\r\nFHx: mom with "thyroid disease," dad with recent heart attcak\r\nAll: none\r\nImmunizations: up to date\r\nSHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms'

In [216]:
features['feature_text'].tolist()[0]

'Family-history-of-MI-OR-Family-history-of-myocardial-infarction'

In [220]:
features

,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded
...,...,...,...
138,912,9,Family-history-of-migraines
139,913,9,Female
140,914,9,Photophobia
141,915,9,No-known-illness-contacts


In [210]:
train.head(30)

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']
5,00016_005,0,16,5,[],[]
6,00016_006,0,16,6,"['adderall', 'adderrall', 'adderrall']","['321 329', '404 413', '652 661']"
7,00016_007,0,16,7,[],[]
8,00016_008,0,16,8,[],[]
9,00016_009,0,16,9,"['palpitations', 'heart beating/pounding']","['26 38', '96 118']"


In [149]:
merged = merged[['annotation','feature_num']]

In [150]:
merged = merged[merged['annotation']!='']

In [151]:
merged['annotation'] = merged['annotation'].str.replace(':','').str.replace('-',' ').str.replace(',',' ')

In [152]:
merged

,annotation,feature_num
0,dad with recent heart attcak,0
1,NaN,0
2,father heart attack,0
3,Father MI,0
4,Dad MI,0
...,...,...
14295,subjective fever,916
14296,NaN,916
14297,feels warm,916
14298,Felt warm,916


In [36]:
merged.to_csv('/Users/suhancho/data/kaggle/nbme/train.tsv',sep='\t',index=None)

In [81]:
len(set(merged['feature_num']))

143

In [96]:
merged

,annotation,feature_num
0,dad with recent heart attcak,0
2,father heart attack,0
3,Father MI,0
4,Dad MI,0
5,father had acute MI,0
...,...,...
14292,feeling warm,916
14295,subjective fever,916
14297,feels warm,916
14298,Felt warm,916
